## Project 1: Class Scheduling with Mathematical Programming

* **Collaborators**: Isidore Sossa and Chase Scott
* **Resources**: 
    * Nurses Scheduling Problem
* **Project's requirements**:
    * Use data provided through Canvas
    * Generate random list of preferences for each professor according to their qualification, i.e. an integer from 1 to *n*, where *n* is the number of classes the professor is qualified to teach. The higher the number the lesser the preference. (**Done**)
    * Extract out professors who do not teach
    * Classes start at 8:00 AM and end at 6:00 PM and can only start at every half period.
    * One credit hour is equivalent to 50 minutes. E.g. A 3-credit hour class meet for a total of 150 minutes per week.
    * Add two constraints of your chosen


In [1]:
import pandas as pd
from docplex.mp.model import Model
import math
import random

### Data Exploration and Cleaning

In [5]:
## Load data
data_file = 'Data Set – Class Schedule.xlsx'
file_handler = pd.ExcelFile(path_or_buffer=data_file)
file_sheets = sorted(file_handler.sheet_names)

print(f'Sheets : {file_sheets}')

## Load each sheet as a DataFrame
classrooms_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[0])
display(file_sheets[0], classrooms_df.head(5))

colleges_and_depts = pd.read_excel(io=file_handler, sheet_name=file_sheets[1])
display(file_sheets[1], colleges_and_depts.head(5))

course_catalog_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[2])
display(file_sheets[2], course_catalog_df.head(5))

courses_offered = pd.read_excel(io=file_handler, sheet_name=file_sheets[3])
display(file_sheets[3], courses_offered.head(5))

professors_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[4])
display(file_sheets[4], professors_df.head(5))

qualifications_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[5])
display(file_sheets[5], qualifications_df.head(5))

# For reproducibility
random.seed(1345)

Sheets : ['Classrooms', 'Colleges and Departments', 'Course Catalog', 'Courses Offered Fall 2021', 'Professors', 'Qualification']


'Classrooms'

,Building,Building Code,Classroom,Capacity
0,Durland,DU,1029,47
1,Durland,DU,1041,24
2,Durland,DU,1052,60
3,Durland,DU,1061,47
4,Durland,DU,1063,47


'Colleges and Departments'

,College,College Code,Department,Department Code
0,College of Agriculture,CAGRI,Entomology,ENTOM


...

,Department Code,Course Number,Course Name,Sections,Day,Max Enrollement
0,ENTOM,300,Economic Entomology,1,M/W/F,50
1,ENTOM,301,Insects and People,3,TU/TH,50
2,ENTOM,305,Animal Health Entomology,3,TU/TH,50
3,ENTOM,306,Animal Health Entomology Laboratory,1,M/W/F,50
4,ENTOM,350,"Crops, Insects, and Agroecosystems",1,M/W,50


'Professors'

,Department Code,Faculty Name,Workload Credit Hours
0,ENTOM,Frank H. Arthur,6
1,ENTOM,James F. Campbell,9
2,ENTOM,Ming-Shun Chen,9
3,ENTOM,Raymond Cloyd,9
4,ENTOM,Lee Cohnstaedt,9


'Qualification'

,Department Code,Faculty Name,ENTOM 300,ENTOM 301,ENTOM 305,ENTOM 306,ENTOM 350,ENTOM 589,ENTOM 602,ENTOM 621,...,ENTOM 830,ENTOM 835,ENTOM 837,ENTOM 840,ENTOM 849,ENTOM 857,ENTOM 860,ENTOM 875,ENTOM 880,ENTOM 885
0,ENTOM,Frank H. Arthur,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,...,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,ENTOM,James F. Campbell,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0
2,ENTOM,Ming-Shun Chen,NaN,1.0,1.0,1.0,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
3,ENTOM,Raymond Cloyd,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,...,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
4,ENTOM,Lee Cohnstaedt,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,...,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN


In [8]:
## Merge professors and qualifications
result = pd.merge(left=professors_df, right=qualifications_df, on=['Faculty Name', 'Department Code'])
result

,Department Code,Faculty Name,Workload Credit Hours,ENTOM 300,ENTOM 301,ENTOM 305,ENTOM 306,ENTOM 350,ENTOM 589,ENTOM 602,...,ENTOM 830,ENTOM 835,ENTOM 837,ENTOM 840,ENTOM 849,ENTOM 857,ENTOM 860,ENTOM 875,ENTOM 880,ENTOM 885
0,ENTOM,Frank H. Arthur,6,NaN,1.0,1.0,NaN,1.0,NaN,NaN,...,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,ENTOM,James F. Campbell,9,NaN,1.0,1.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0
2,ENTOM,Ming-Shun Chen,9,NaN,1.0,1.0,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
3,ENTOM,Raymond Cloyd,9,NaN,1.0,1.0,1.0,1.0,1.0,NaN,...,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
4,ENTOM,Lee Cohnstaedt,9,1.0,1.0,1.0,1.0,1.0,NaN,1.0,...,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN
5,ENTOM,Srinivas Kambhampati,6,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN
6,ENTOM,Tania Kim,3,1.0,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ENTOM,Berlin Luxelly Londono Renteria,6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ENTOM,Jeremy L. Marshall,9,1.0,1.0,1.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ENTOM,Brian P. McCornack,9,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN


### Functions

In [17]:
## Clean
def get_preference(qualifications:pd.DataFrame, course:str) -> pd.DataFrame:
    ''''''
    def manipulate_row(row:pd.DataFrame, course:str):
        cols = [x for x in row.columns if course in x]
        class_taught = 0
        for col in cols:
            if not math.isnan(row[col][0]):
                class_taught += row[col][0]
        class_taught = int(class_taught)
        if row['Workload Credit Hours'][0] > 0:
            preferences = random.sample(list(range(1, class_taught + 1)), class_taught)
        else:
            preferences = [0] * class_taught
            
        #display(preferences)

        index = 0
        for col in cols:
            if not math.isnan(row[col][0]):
                row[col] = preferences[index]
                index += 1
            else:
                row[col] = 0
        return row

    col_names = list(qualifications.columns)
    preferences_df = pd.DataFrame(columns=col_names)
    col_names = qualifications.columns
    #preferences_df = qualifications.apply(lambda row : )
    for row in qualifications.itertuples():
        row_df = pd.Series(row).to_frame().T.drop(columns=0)
        row_df.columns = col_names
        print(row_df)
        row_df = manipulate_row(row_df, course)
        preferences_df = preferences_df.append(row_df, ignore_index=True)
    return preferences_df


## Merge professors and qualifications
result = pd.merge(left=professors_df, right=qualifications_df, on=['Faculty Name', 'Department Code'])
result
preferences_df = get_preference(result, 'ENTOM')
display(preferences_df)

  Department Code     Faculty Name Workload Credit Hours ENTOM 300 ENTOM 301  \
0           ENTOM  Frank H. Arthur                     6       NaN       1.0   

  ENTOM 305 ENTOM 306 ENTOM 350 ENTOM 589 ENTOM 602  ... ENTOM 830 ENTOM 835  \
0       1.0       NaN       1.0       NaN       NaN  ...       1.0       NaN   

  ENTOM 837 ENTOM 840 ENTOM 849 ENTOM 857 ENTOM 860 ENTOM 875 ENTOM 880  \
0       NaN       1.0       1.0       NaN       NaN       NaN       NaN   

  ENTOM 885  
0       NaN  

[1 rows x 37 columns]
  Department Code       Faculty Name Workload Credit Hours ENTOM 300  \
0           ENTOM  James F. Campbell                     9       NaN   

  ENTOM 301 ENTOM 305 ENTOM 306 ENTOM 350 ENTOM 589 ENTOM 602  ... ENTOM 830  \
0       1.0       1.0       NaN       1.0       1.0       NaN  ...       NaN   

  ENTOM 835 ENTOM 837 ENTOM 840 ENTOM 849 ENTOM 857 ENTOM 860 ENTOM 875  \
0       NaN       NaN       NaN       1.0       NaN       NaN       NaN   

  ENTOM 880 ENTOM 8

  Department Code              Faculty Name Workload Credit Hours ENTOM 300  \
0           ENTOM  Robert Jeffery Whitworth                     0       1.0   

  ENTOM 301 ENTOM 305 ENTOM 306 ENTOM 350 ENTOM 589 ENTOM 602  ... ENTOM 830  \
0       1.0       1.0       1.0       NaN       1.0       NaN  ...       1.0   

  ENTOM 835 ENTOM 837 ENTOM 840 ENTOM 849 ENTOM 857 ENTOM 860 ENTOM 875  \
0       NaN       1.0       NaN       NaN       NaN       NaN       NaN   

  ENTOM 880 ENTOM 885  
0       NaN       NaN  

[1 rows x 37 columns]
  Department Code Faculty Name Workload Credit Hours ENTOM 300 ENTOM 301  \
0           ENTOM  Kun Yan Zhu                     3       1.0       1.0   

  ENTOM 305 ENTOM 306 ENTOM 350 ENTOM 589 ENTOM 602  ... ENTOM 830 ENTOM 835  \
0       NaN       1.0       1.0       NaN       1.0  ...       NaN       NaN   

  ENTOM 837 ENTOM 840 ENTOM 849 ENTOM 857 ENTOM 860 ENTOM 875 ENTOM 880  \
0       1.0       NaN       NaN       NaN       NaN       NaN       N

,Department Code,Faculty Name,Workload Credit Hours,ENTOM 300,ENTOM 301,ENTOM 305,ENTOM 306,ENTOM 350,ENTOM 589,ENTOM 602,...,ENTOM 830,ENTOM 835,ENTOM 837,ENTOM 840,ENTOM 849,ENTOM 857,ENTOM 860,ENTOM 875,ENTOM 880,ENTOM 885
0,ENTOM,Frank H. Arthur,6,0,8,2,0,9,0,0,...,4,0,0,6,7,0,0,0,0,0
1,ENTOM,James F. Campbell,9,0,8,7,0,6,3,0,...,0,0,0,0,5,0,0,0,1,4
2,ENTOM,Ming-Shun Chen,9,0,3,8,5,0,7,0,...,0,0,0,0,0,0,0,9,4,1
3,ENTOM,Raymond Cloyd,9,0,8,7,4,1,6,0,...,0,2,0,0,9,5,0,0,0,0
4,ENTOM,Lee Cohnstaedt,9,9,2,6,7,5,0,11,...,8,0,0,0,1,0,3,0,0,0
5,ENTOM,Srinivas Kambhampati,6,0,0,9,2,0,0,0,...,0,0,0,1,0,0,4,0,7,0
6,ENTOM,Tania Kim,3,7,0,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,ENTOM,Berlin Luxelly Londono Renteria,6,5,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0
8,ENTOM,Jeremy L. Marshall,9,3,6,7,4,0,0,8,...,0,0,0,0,0,0,0,0,0,0
9,ENTOM,Brian P. McCornack,9,0,0,0,8,0,0,0,...,5,2,0,7,3,0,0,0,0,0


### Mathematical Model

In [4]:
## Model Environment
model = Model(name='Course Scheduling')
model.print_information()

## Decision Variables

## Constraints

## Objective Function


Model: Course Scheduling
 - number of variables: 0
   - binary=0, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: LP


In [9]:
[0]*5

[0, 0, 0, 0, 0]